In [28]:
import requests as rq
import pandas as pd
import re
from tqdm import tqdm
import sqlite3
import time
import os.path
import ast
from HH_api import hh_token as ad

In [29]:
api_hh = 'https://api.hh.ru/'
date_from = '1970-01-02'
page = 0
per_page = 100
limit = 2000
comp_name = 'сбер'
token = ad.hh_token_type + ' ' + ad.hh_token

In [30]:
# надо будет перенести в начало кода, так как в него же будем подгружать данные из CSV
ls_columns = ['id', 'description', 'key_skills', 'schedule', 'experience', 'code', 'department'
    , 'employment', 'salary', 'archived', 'name', 'area', 'created_at', 'published_at'
    , 'employer', 'type', 'has_test', 'test', 'specialization', 'contacts', 'working_days'
    , 'working_time_intervals', 'working_time_modes', 'accept_temporary']
vacancies = pd.DataFrame(columns=ls_columns)
# ,id,description,key_skills,schedule,experience,code,department
# ,employment,salary,archived,name,area,created_at,published_at
# ,employer,type,has_test,test,specialization,contacts,working_days
# ,working_time_intervals,working_time_modes,accept_temporary
ser_vacs_id_old = pd.Series()

# Нужно проверить наличие файла выгрузки:
if os.path.exists('HH_api/data/download_source.csv'):
    print('Найден файл выгрузки. Будем проверять наличие новых появившихся вакансий, а также удалим закрытые вакансии.')
    vacancies = pd.read_csv('HH_api/data/download_source.csv')
    vacancies = vacancies.drop('Unnamed: 0', axis=1)
    df_vacs_id_old = vacancies[['id']].rename(columns={'id':'id_old'})

# df_vacs_id_old

<ipython-input-30-f3f23dc010ce>:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  ser_vacs_id_old = pd.Series()


Найден файл выгрузки. Будем проверять наличие новых появившихся вакансий, а также удалим закрытые вакансии.


In [31]:
vacs_comp = pd.DataFrame(rq.get(api_hh+'employers?text='+comp_name, headers={'authorization': token})
                         .json().get('items'))

vacs_comp = vacs_comp[vacs_comp.open_vacancies != 0].sort_values(by='open_vacancies', ascending=False)
ls_comp_id = vacs_comp.id.to_list()
ls_comp_id[0]

'3529'

In [32]:
filter_vacs_dict = {'Authorization':token, 'per_page':per_page, 'page':page, 'date_from':date_from
    , 'order_by':'publication_time', }

In [33]:
df_log = pd.DataFrame(columns={'hyperlink', 'vacation_id', 'rq_alt_url'})

vacs_id = []
for comp_iter in [ls_comp_id[0]]:
#     добавляем в словарь параметров идентификатор компании, по которой совершаем поиск
    filter_vacs_dict['employer_id'] = comp_iter
#     формируем запрос к АПИ на получение списка вакансий и тех. данные поиска
    rq_api_hh = rq.get(api_hh+'vacancies?', params=filter_vacs_dict).json()
#     записываем в переменную количество найденных вакансий по указанному фильтру
    quantity_vacs = rq_api_hh.get('found')
    print('Всего найдено вакансий: '+str(quantity_vacs))
    print(rq_api_hh.get('alternate_url'))
#     из полученного ответа от АПИ формируем DataFrame с идентификаторы
#     вакансий нулевой страницы и датой создания вакансии
    df_test = pd.DataFrame(rq_api_hh.get('items'))[['id', 'created_at']].sort_values(by='created_at')
#     записываем идентификаторы в list, чтобы по ним потом пробегаться
    vacs_id = vacs_id + df_test.id.to_list()
#     записываем максимальную дату создания, чтобы от нее в дальнейшем вести отсчет
    date_to = df_test['created_at'].loc[0]
    print(date_to)
#     если вакансий больше, чем может вернуть АПИ
    filter_vacs_dict['date_to'] = date_to[0:10]
    while quantity_vacs > limit:
        filter_vacs_dict['page'] = 0
        rq_api_hh = rq.get(api_hh+'vacancies?', params=filter_vacs_dict).json()
        df_test = pd.DataFrame(rq_api_hh.get('items'))[['id', 'created_at']].sort_values(by='created_at')
        count_pages = rq_api_hh.get('pages')
        print(rq_api_hh.get('alternate_url'))
        vacs_id = vacs_id + df_test.id.to_list()
    #         забираем данные с 20 страниц
        for pg in range(count_pages-1):
    #             обновляем номер страницы в параметрах, прибавляя единицу, т.к.
    #             данные нулевой страницы у нас уже имеются
            filter_vacs_dict['page'] = pg + 1
    #             формируем запрос к АПИ на получение списка вакансий и тех. данные поиска
            rq_api_hh = rq.get(api_hh+'vacancies?', params=filter_vacs_dict).json()
    #             из полученного ответа от АПИ формируем DataFrame с идентификаторы
    #             вакансий нулевой страницы и датой создания вакансии
            df_test = pd.DataFrame(rq_api_hh.get('items'))[['id', 'created_at']].sort_values(by='created_at')
    #             записываем идентификаторы в list, чтобы по ним потом пробегаться
            vacs_id = vacs_id + df_test.id.to_list()


#         записываем в переменную количество найденных вакансий по указанному фильтру
        quantity_vacs = rq_api_hh.get('found')
        print(quantity_vacs)
#         записываем максимальную дату создания, чтобы от нее в дальнейшем вести отсчет
        date_to = df_test['created_at'].loc[0]
        filter_vacs_dict['date_to'] = date_to[0:10]

# так как выгружаем по датам - нахлестом на один день, то будут дубли, удаляем их
vacs_id = pd.Series(pd.Series(vacs_id).unique()).tolist()

df_vacs_id_new = pd.DataFrame(pd.Series(pd.to_numeric(pd.Series(vacs_id).unique())), columns=['id_new'])

Всего найдено вакансий: 5938
https://hh.ru/search/vacancy?date_from=02.01.1970&employer_id=3529&enable_snippets=true&items_on_page=100&order_by=publication_time
2021-12-10T11:50:55+0300
https://hh.ru/search/vacancy?date_from=02.01.1970&date_to=10.12.2021+23%3A59%3A59&employer_id=3529&enable_snippets=true&items_on_page=100&order_by=publication_time
5939
https://hh.ru/search/vacancy?date_from=02.01.1970&date_to=03.12.2021+23%3A59%3A59&employer_id=3529&enable_snippets=true&items_on_page=100&order_by=publication_time
4120
https://hh.ru/search/vacancy?date_from=02.01.1970&date_to=25.11.2021+23%3A59%3A59&employer_id=3529&enable_snippets=true&items_on_page=100&order_by=publication_time
2392
https://hh.ru/search/vacancy?date_from=02.01.1970&date_to=15.11.2021+23%3A59%3A59&employer_id=3529&enable_snippets=true&items_on_page=100&order_by=publication_time
660


In [34]:
if os.path.exists('HH_api/data/download_source.csv'):
    # исключаем закрытые вакансии
    df_vacs_id_old = df_vacs_id_old[(pd.merge(df_vacs_id_old, df_vacs_id_new,
                                              how='left', right_on='id_new', left_on='id_old'))['id_new'].notna()]

    vacancies = vacancies[pd.merge(vacancies, df_vacs_id_old, how='left'
                                   , left_on='id', right_on='id_old')['id_old'].notna()]
    vacancies = vacancies.reset_index().drop(columns='index')

    # оставляем только вакансии, которых не было в сохраненной выгрухке из файла
    vacs_id = df_vacs_id_new[pd.merge(df_vacs_id_old, df_vacs_id_new,
                                      how='right', right_on='id_new',
                                      left_on='id_old')['id_old'].isna()]['id_new'].astype(str).to_list()

print(len(vacs_id))
vacancies

9


,id,description,key_skills,schedule,experience,code,department,employment,salary,archived,...,employer,type,has_test,test,specialization,contacts,working_days,working_time_intervals,working_time_modes,accept_temporary
0,44326135,<p><strong>Если ты:</strong></p> <p> </p> <ul>...,[],"{'id': 'fullDay', 'name': 'Полный день'}","{'id': 'between1And3', 'name': 'От 1 года до 3...",NaN,"{'id': '3529-3529-stat', 'name': 'Сбер. Начало...","{'id': 'full', 'name': 'Полная занятость'}","{'from': 33000, 'to': None, 'currency': 'RUR',...",False,...,"{'id': '3529', 'name': 'Сбербанк', 'url': 'htt...","{'id': 'direct', 'name': 'Рекламная'}",False,NaN,NaN,NaN,[],[],[],False
1,44326087,<p>Задача клиентского менеджера ― продавать пр...,[],"{'id': 'fullDay', 'name': 'Полный день'}","{'id': 'noExperience', 'name': 'Нет опыта'}",NaN,"{'id': '3529-3529-stat', 'name': 'Сбер. Начало...","{'id': 'full', 'name': 'Полная занятость'}","{'from': 35900, 'to': None, 'currency': 'RUR',...",False,...,"{'id': '3529', 'name': 'Сбербанк', 'url': 'htt...","{'id': 'direct', 'name': 'Рекламная'}",False,NaN,NaN,NaN,[],[],[],False
2,44326154,<p>Задача клиентского менеджера ― продавать пр...,"[{'name': 'Активные продажи'}, {'name': 'Креди...","{'id': 'fullDay', 'name': 'Полный день'}","{'id': 'noExperience', 'name': 'Нет опыта'}",NaN,"{'id': '3529-3529-stat', 'name': 'Сбер. Начало...","{'id': 'full', 'name': 'Полная занятость'}","{'from': 35000, 'to': None, 'currency': 'RUR',...",False,...,"{'id': '3529', 'name': 'Сбербанк', 'url': 'htt...","{'id': 'direct', 'name': 'Рекламная'}",False,NaN,NaN,NaN,[],[],[],False
3,44326053,<p>Задача клиентского менеджера ― продавать пр...,"[{'name': 'Активные продажи'}, {'name': 'Прямы...","{'id': 'fullDay', 'name': 'Полный день'}","{'id': 'noExperience', 'name': 'Нет опыта'}",NaN,"{'id': '3529-3529-clients', 'name': 'Сбер. Раб...","{'id': 'full', 'name': 'Полная занятость'}","{'from': 42500, 'to': None, 'currency': 'RUR',...",False,...,"{'id': '3529', 'name': 'Сбербанк', 'url': 'htt...","{'id': 'direct', 'name': 'Рекламная'}",False,NaN,NaN,NaN,[],[],[],False
4,44326143,<p>Задача клиентского менеджера ― продавать пр...,"[{'name': 'Активные продажи'}, {'name': 'Креди...","{'id': 'fullDay', 'name': 'Полный день'}","{'id': 'noExperience', 'name': 'Нет опыта'}",NaN,"{'id': '3529-3529-stat', 'name': 'Сбер. Начало...","{'id': 'full', 'name': 'Полная занятость'}","{'from': 29700, 'to': None, 'currency': 'RUR',...",False,...,"{'id': '3529', 'name': 'Сбербанк', 'url': 'htt...","{'id': 'direct', 'name': 'Рекламная'}",False,NaN,NaN,NaN,[],[],[],False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5920,50351941,<p><strong>Клиентский менеджер малого и микроб...,"[{'name': 'Деловая переписка'}, {'name': 'Обуч...","{'id': 'fullDay', 'name': 'Полный день'}","{'id': 'between1And3', 'name': 'От 1 года до 3...",NaN,"{'id': '3529-3529-prof', 'name': 'Сбер для экс...","{'id': 'full', 'name': 'Полная занятость'}","{'from': 35900, 'to': None, 'currency': 'RUR',...",False,...,"{'id': '3529', 'name': 'Сбербанк', 'url': 'htt...","{'id': 'open', 'name': 'Открытая'}",False,NaN,NaN,"{'name': 'Никитина Ксения', 'email': 'kynikiti...",[],[],[],False
5921,50351980,<p>Менеджер по продаже зарплатных проектов — в...,[],"{'id': 'fullDay', 'name': 'Полный день'}","{'id': 'between1And3', 'name': 'От 1 года до 3...",NaN,"{'id': '3529-3529-prof', 'name': 'Сбер для экс...","{'id': 'full', 'name': 'Полная занятость'}","{'from': 80000, 'to': None, 'currency': 'RUR',...",False,...,"{'id': '3529', 'name': 'Сбербанк', 'url': 'htt...","{'id': 'open', 'name': 'Открытая'}",False,NaN,NaN,"{'name': 'Команда подбора персонала Сбера', 'e...",[],[],[],False
5922,50352016,<p><b>Доход 45 600 руб до вычета НДФЛ</b></p><...,[],"{'id': 'fullDay', 'name': 'Полный день'}","{'id': 'between1And3', 'name': 'От 1 года до 3...",NaN,"{'id': '3529-3529-prof', 'name': 'Сбер для экс...","{'id': 'full', 'name': 'Полная занятость'}","{'from': 45600, 'to': None, 'curr

In [35]:
ls_errors = []
# DF в который записываем с каким кодом получили данные по вакансии
df_log = pd.DataFrame(columns={'hyperlink', 'vacation_id', 'rq_alt_url'})

if len(vacs_id) != 0:
    for i in tqdm(range(len(vacs_id))):
        rq_api_hh_vac = rq.get('https://api.hh.ru/vacancies/' + vacs_id[i]
                               , headers={list(filter_vacs_dict.keys())[0]: list(filter_vacs_dict.values())[0]})
        # если получаем ошибку 403, то нас отрубают, так как слишком много запросов отправляем
        # поэтому ждем 5 минут и пробуем снова получить данные по вакансии пока не получим код 200
        # while rq_api_hh_vac.status_code == 403:
        if rq_api_hh_vac.status_code == 403:
            print('https://api.hh.ru/vacancies/' + vacs_id[i])
            ls_errors.append(rq_api_hh_vac)
            # print(rq_api_hh_vac.json())
            print('Получили код ошибки 403. Ждем 5 минут и отправляем запрос заново.')
            # time.sleep(5*60)
            # rq_api_hh_vac = rq.get('https://api.hh.ru/vacancies/' + vacs_id[i]
            #                        , headers={list(filter_vacs_dict.keys())[0]: list(filter_vacs_dict.values())[0]})
            continue
        # если получаем код ответа 200, то записываем в DF данные вакансии
        if rq_api_hh_vac.status_code == 502:
            print('https://api.hh.ru/vacancies/' + vacs_id[i])
            print(rq_api_hh_vac.json())
            print('Получили код ошибки 502. Ждем 10 сек и отправляем запрос заново.')
            time.sleep(10)
            rq_api_hh_vac = rq.get('https://api.hh.ru/vacancies/' + vacs_id[i]
                                   , headers={list(filter_vacs_dict.keys())[0]: list(filter_vacs_dict.values())[0]})
        if rq_api_hh_vac.status_code == 200:
            vacancies.loc[len(vacancies)] = [rq_api_hh_vac.json().get(it) for it in ls_columns]
        else:
            # иначе выводим в лог идентификатор вакансии и код ошибки от api
            print('Вакансия с ID: ' + str(vacs_id[i]) + '; получили ошибку: ' + str(rq_api_hh_vac.status_code))
            continue
        # записываем лог
        df_log.loc[len(df_log)] = ['https://api.hh.ru/vacancies/' + vacs_id[i]
            , vacs_id[i]
            , rq_api_hh_vac.json().get('alternate_url')]
    # выгружаем данные в файл, чтобы не грузить api огромным количеством запросов каждый запуск
    vacancies.to_csv('HH_api/data/download_source.csv')
else:
    print('Новых вакансий не обнаружено')

100%|██████████| 9/9 [00:02<00:00,  3.41it/s]


In [36]:
df_parsed_vacancies = vacancies.copy()

#  парсим выгрузку в df
# убираме HTML символы из описания вакансий
df_parsed_vacancies.description = df_parsed_vacancies.description.apply(lambda x: re.sub(r"\<[^>]*\>", '', x))
df_parsed_vacancies.description = df_parsed_vacancies.description.apply(lambda x: str(x).replace('\r\n', ''))

df_parsed_vacancies.key_skills = df_parsed_vacancies.key_skills.astype(str)
df_parsed_vacancies.key_skills = df_parsed_vacancies.key_skills.apply(lambda x: ast.literal_eval(x))
df_parsed_vacancies.key_skills = df_parsed_vacancies.key_skills.\
    apply(lambda x: '; '.join([list(x[i].values())[0] for i in range(len(x))]))

ls_cols_pars = ['schedule', 'experience', 'department', 'employment']

for i in ls_cols_pars:
    df_parsed_vacancies[i] = df_parsed_vacancies[i].astype(str)
    df_parsed_vacancies[i] = df_parsed_vacancies[i].apply(lambda x: ast.literal_eval(x))
    df_parsed_vacancies[i] = df_parsed_vacancies[i].apply(lambda x: list(x.values())[1])


for col_name in ['salary', 'area', 'employer', 'type']:
    df_parsed_vacancies[col_name] = df_parsed_vacancies[col_name].astype(str)
    df_parsed_vacancies['temp_len'] = 0
    df_parsed_vacancies['temp_name_exist'] = 0
    df_parsed_vacancies[col_name + '_str'] = 'nan'
    for i in df_parsed_vacancies[df_parsed_vacancies[col_name] != 'nan'].index.to_list():
        df_parsed_vacancies['temp_len'].iloc[i] = len(ast.literal_eval(df_parsed_vacancies[col_name].iloc[i]))
        df_parsed_vacancies['temp_name_exist'].iloc[i] = \
            [1 if list(ast.literal_eval(df_parsed_vacancies[col_name].iloc[i]).keys()).count('name') > 0 else 0][0]
        if df_parsed_vacancies['temp_name_exist'].max() > 0:
            df_parsed_vacancies[col_name + '_str'].iloc[i] = ast.literal_eval(df_parsed_vacancies[col_name].iloc[i]).\
                get('name')
        else:
            temp_dict_values = ast.literal_eval(df_parsed_vacancies[col_name].iloc[i])
            key_names = list(temp_dict_values.keys())
            temp_string = 'от '+ str(temp_dict_values.get(key_names[0])) + \
                ' до ' + str(temp_dict_values.get(key_names[1])) + ' ' + str(temp_dict_values.get(key_names[2])) + \
                ' ' + str(temp_dict_values.get(key_names[3]))
            temp_string = (((temp_string.replace('до None', '')).replace('RUR', 'руб.')).replace('True', 'гросс')).\
                replace('nan', '')
            df_parsed_vacancies[col_name + '_str'].iloc[i] = temp_string


df_parsed_vacancies = df_parsed_vacancies.\
    drop(columns=['salary', 'area', 'employer', 'type' \
    , 'temp_len', 'temp_name_exist', 'specialization', \
                  'working_days', 'working_time_intervals', 'working_time_modes', 'accept_temporary'])


df_parsed_vacancies['contacts'] = df_parsed_vacancies['contacts'].astype(str)
df_parsed_vacancies['hr'] = 'nan'
df_parsed_vacancies['hr_email'] = 'nan'
df_parsed_vacancies['hr_phones'] = 'nan'
for i in range(len(df_parsed_vacancies[df_parsed_vacancies['contacts'] != 'nan'])):
    temp_contacts = ast.literal_eval(df_parsed_vacancies[df_parsed_vacancies['contacts'] != 'nan']['contacts'].iloc[i])
    temp_phones = temp_contacts.pop('phones')
    temp_all_phones = []
    if len(temp_phones) != 0:
        for tp_i in range(len(temp_phones)):
            if len(temp_phones[tp_i]) == 0:
                continue
            else:
                temp_all_phones.append('+' + str(temp_phones[tp_i].pop('country')).replace('None', '') \
                              + str(temp_phones[tp_i].pop('city')).replace('None', '') \
                              + str(temp_phones[tp_i].pop('number')).replace('None', '')
                              + (' ' + str(temp_phones[tp_i].pop('comment'))).replace(' None', ''))
    # else: temp_phones = 'nan'
    if len(temp_all_phones) == 0: temp_all_phones = 'nan'
    temp_hr = temp_contacts.pop('name')
    temp_email = temp_contacts.pop('email')
    df_parsed_vacancies['hr'].iloc[i] = temp_hr
    df_parsed_vacancies['hr_email'].iloc[i] = temp_email
    df_parsed_vacancies['hr_phones'].iloc[i] = temp_all_phones
df_parsed_vacancies = df_parsed_vacancies.drop(columns=['contacts'])

df_parsed_vacancies.to_csv('HH_api/data/result.csv', encoding='cp1251', errors='ignore')


C:\Users\iKerng\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
